# Домашнее задание №2: линейная регрессия (10 баллов).

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np
from numpy import linalg as LA

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(10000, 100) (10000, 1)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
mean_squared_error(y, reg.predict(X))

1.707155037524044e-25

Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [4]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor().fit(X, y)
mean_squared_error(y, reg.predict(X))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.00041907276059434883

Как видим значение близко к LinearRegression

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

Разница вызвана наличием регуляризации в SGD и тем, что SGD считает градиент по случайному наблюдению в то время как LinearRegresssion усредняет всю выборку. Но + SGD в его скорости.

In [5]:
reg1 = SGDRegressor(alpha = 0, tol = 1e-9, max_iter = 100000, learning_rate = 'constant', eta0 = 0.001).fit(X, y)
mean_squared_error(y, reg1.predict(X)) # В SGDRegressor по дефолту стоит регуляризация и если обнулить альфу, то это будет обычная регрессия


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2.7145737384275128e-25

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию. 

In [6]:
X.shape, y.shape

((10000, 100), (10000, 1))

In [7]:
X_new = np.concatenate([X, np.ones(X.shape[0]).reshape(-1, 1)], axis = 1) # В векторе Х нет константы, создадим ее

In [8]:
class LinearRegression(object):
    def __init__(self, alpha=0.01, l_ratio=0.001, tol=0.0001, max_iter=1000, lr_decay = 1, regularization = None, seed = 1):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.regularization = regularization
        self.lr_decay = lr_decay

    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        self.X = X
        self.y = y
        self.num_iter = 0
        self.w_0 = np.random.normal(0, 1, (self.X.shape[1], 1)) # инициализируем веса из нормального распределия N(0, 1)
        self.diff = 100000 # Инициализация для критерия останова
        
        #np.random.seed(123)
            
        if self.regularization == None:
            for i in range(self.max_iter): # Ограничение на число итераций
                if self.diff > self.tol: # Критерий останова
                    self.alpha = self.alpha * self.lr_decay # Адаптивный шаг градинта

                    self.w_new = self.w_0 - self.alpha*2/self.X.shape[0]*np.dot(self.X.T, np.dot(self.X, self.w_0) - self.y) # Обновление градиента
            
                    self.diff =  LA.norm(self.w_new - self.w_0) # Подсчет нормы разности весов

                    self.w_0 = self.w_new # Обновление W(k-1) для шага k
                    self.num_iter += 1
        elif self.regularization == 'l2':
            for i in range(self.max_iter): # Ограничение на число итераций
                if self.diff > self.tol: # Критерий останова
                    self.alpha = self.alpha * self.lr_decay # Адаптивный шаг градинта
             
                    self.w_new = self.w_0 - self.alpha*2/self.X.shape[0]*(np.dot(self.X.T, np.dot(self.X, self.w_0) - self.y) + self.l_ratio*self.w_0)
                    # Обновление градиента c учетом L2-регуляризации
            
                    self.diff =  LA.norm(self.w_new - self.w_0) # Подсчет нормы разности весов

                    self.w_0 = self.w_new # Обновление W(k-1) для шага k
                    self.num_iter += 1

        elif self.regularization == 'l1':
            for i in range(self.max_iter): # Ограничение на число итераций
                if self.diff > self.tol: # Критерий останова
                    self.alpha = self.alpha * self.lr_decay # Адаптивный шаг градинта
             
                    self.w_new = self.w_0 - self.alpha*2/self.X.shape[0]*np.dot(self.X.T, np.dot(self.X, self.w_0) - self.y) 
                    + self.l_ratio*np.sign(self.w_0)
                    # Обновление градиента c учетом L1-регуляризации
            
                    self.diff =  LA.norm(self.w_new - self.w_0) # Подсчет нормы разности весов

                    self.w_0 = self.w_new # Обновление W(k-1) для шага k
                    self.num_iter += 1

                      
        return self.w_new 
            
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        self.y_pred = np.dot(self.X, self.w_new) # Предсказание модели
        
        return self.y_pred
        # your code here

In [9]:
# Linear regression
my_reg = LinearRegression(regularization = None, lr_decay = 1)
my_reg.fit(X_new, y)
assert mean_squared_error(y, my_reg.predict(X_new)) < 1e-3
print('You are amazing! Great work!')
mean_squared_error(y, my_reg.predict(X_new))

You are amazing! Great work!


2.6696364683813824e-05

In [10]:
# Lasso regression
my_reg = LinearRegression(regularization = 'l1', lr_decay = 1, l_ratio = 0.1)
my_reg.fit(X_new, y)
assert mean_squared_error(y, my_reg.predict(X_new)) < 1e-3
print('You are amazing! Great work!')
mean_squared_error(y, my_reg.predict(X_new))

You are amazing! Great work!


2.71100584567059e-05

In [11]:
# Ridge regression
my_reg = LinearRegression(regularization = 'l2', lr_decay =0.999, l_ratio = 0.1)
my_reg.fit(X_new, y)
assert mean_squared_error(y, my_reg.predict(X_new)) < 1e-3
print('You are amazing! Great work!')
mean_squared_error(y, my_reg.predict(X_new))

You are amazing! Great work!


0.00016121958825897359

***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый вариант) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй вариант) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [12]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.1).fit(X, y)
mean_squared_error(y, lasso_reg.predict(X))

0.09864602735396616

In [13]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.1).fit(X, y)
mean_squared_error(y, ridge_reg.predict(X))

4.267555678619991e-06

Результаты написанной модели и моделей из коробки - похожие. В Lasso из коробки нельзя менять шаг спуска. В написанной модели есть адаптивный шаг спуска. Кроме этого я инициализирую веса изи нормального распределения для регрессии, Lasso, Ridge. Как это делается в коробке не нашел

Для Lasso из коробки MSE гораздо выше, чем для Ridge. Хотя Ridge вроде как сильнее штрафует за большие веса. C другой стороны Lasso хорошо работает в случае мультиколлинеарности, зануляя веса, что сильно может понизить MSE.